INSTALL THE FOLLOWING PYTHON PACKAGES FIRST BEFORE RUNNING THE PROGRAM

1) Numpy
2) NNFS - for the Spiral dataset
3) scikit-learn - for the iris dataset

In [53]:
# Library imports
import numpy as np

Create classes for modularity

In [56]:
# Hidden Layers
# Dense
class Layer_Dense:
    # Layer initialization
    # randomly initialize weights and set biases to zero
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))


    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs, weight and biases
        self.output = np.dot(inputs, self.weights) + self.biases

    # Backward pass/Backpropagation
    def backward(self, dvalues):
        # Gradients on parameters:
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        # Gradient on values
        self.dinputs = np.dot(dvalues, self.weights.T)


In [58]:
# Activation Functions
# Included here are the functions for both the forward and backward pass

# Linear
class ActivationLinear:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs

    def backward(self, dvalues):
        self.dinputs = dvalues.copy()

# Sigmoid
class ActivationSigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))

    def backward(self, dvalues):
        self.dinputs = dvalues * (self.output * (1 - self.output))

# TanH
class ActivationTanH:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.tanh(inputs)

    def backward(self, dvalues):
        self.dinputs = dvalues * (1 - self.output ** 2)

# ReLU
class Activation_ReLU:
    # Forward pass
    def forward(self, inputs):
        # Remember the input values
        self.inputs = inputs
        # Calculate the output values from inputs
        self.output = np.maximum(0, inputs)

    # Backward pass
    def backward(self, dvalues):
        # Make a copy of the original values first
        self.dinputs = dvalues.copy()
    
        # Zero gradient where input values were negative
        self.dinputs[self.inputs <= 0] = 0

# Softmax
class Activation_Softmax:
    # Forward pass
    def forward(self, inputs):
        # Remember the inputs values
        self.inputs = inputs

        # Get the unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))

        # Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)

        self.output = probabilities

    # Backward pass
    def backward(self, dvalues):
        # Create uninitialized array
        self.dinputs = np.empty_like(dvalues)

        # Enumerate outputs and gradients
        for index, (single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):

            # Flatten output array
            single_output = single_output.reshape(-1, 1)
            # Calculate Jacobian matrix of the output
            jacobian_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # Calculate the sample-wise gradient
            # and add it to the array of sample gradients
            self.dinputs[index] = np.dot(jacobian_matrix, single_dvalues)

In [60]:
# Loss functions

class Loss:
    # Calculate the data and regularization losses
    # Given the model output and grou truth/target values
    def calculate(self, output, y):
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        # Calculate the mean loss
        data_loss = np.mean(sample_losses)
        # Return the mean loss
        return data_loss

# MSE
class Loss_MSE:
    def forward(self, y_pred, y_true):
        # Calculate Mean Squared Error
        return np.mean((y_true - y_pred) ** 2, axis=-1)

    def backward(self, y_pred, y_true):
        # Gradient of MSE loss
        samples = y_true.shape[0]
        outputs = y_true.shape[1]
        self.dinputs = -2 * (y_true - y_pred) / outputs
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Binary Cross-Entropy
class Loss_BinaryCrossEntropy:
    def forward(self, y_pred, y_true):
        # Clip predictions
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        # Calculate Binary Cross Entropy
        return -(y_true * np.log(y_pred_clipped) + (1 - y_true) * np.log(1 - y_pred_clipped))

    def backward(self, y_pred, y_true):
        # Gradient of BCE loss
        samples = y_true.shape[0]
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        self.dinputs = - (y_true / y_pred_clipped - (1 - y_true) / (1 - y_pred_clipped))
        # Normalize gradients over samples
        self.dinputs = self.dinputs / samples

# Categorical Cross-Entropy
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = y_pred.shape[0]

        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Probabilities for target values
        # Only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

    # Backward pass
    def backward(self, dvalues, y_true):
        # Number of samples
        samples = len(dvalues)
        # Number of labels in every sample
        # Use the first sample to count them
        labels = len(dvalues[0])

        # Check if labels are sparse, turn them into one-hot vector values
        # the eye function creates a 2D array with ones on the diagonal and zeros elsewhere
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]

        # Calculate the gradient
        self.dinputs = -y_true / dvalues
        self.dinputs = self.dinputs / samples


<!-- Star -->

In [67]:
class Optimizer_SGD:
    def __init__(self, learning_rate=1.0, decay=0.0, momentum=0.0, adaptive=False, epsilon=1e-7, beta1=0.9, beta2=0.999):
        self.learning_rate = learning_rate
        self.current_lr = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        self.adaptive = adaptive
        self.epsilon = epsilon
        self.beta1 = beta1
        self.beta2 = beta2

    # ---------- Apply learning rate decay ----------
    def pre_update_lr(self):
        if self.decay:
            self.current_lr = self.learning_rate * (1.0 / (1.0 + self.decay * self.iterations))

    # ---------- Update weights ----------
    def update_params(self, layer):
        # Vanilla SGD + momentum
        if self.momentum:
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)

            weight_update = self.momentum * layer.weight_momentums - self.current_lr * layer.dweights
            bias_update = self.momentum * layer.bias_momentums - self.current_lr * layer.dbiases

            layer.weight_momentums = weight_update
            layer.bias_momentums = bias_update
        else:
            weight_update = -self.current_lr * layer.dweights
            bias_update = -self.current_lr * layer.dbiases

        # Adaptive gradient
        if self.adaptive:
            if not hasattr(layer, 'weight_cache'):
                layer.weight_moment = np.zeros_like(layer.weights)
                layer.bias_moment = np.zeros_like(layer.biases)
                layer.weight_cache = np.zeros_like(layer.weights)
                layer.bias_cache = np.zeros_like(layer.biases)

            layer.weight_moment = self.beta1 * layer.weight_moment + (1 - self.beta1) * layer.dweights
            layer.bias_moment = self.beta1 * layer.bias_moment + (1 - self.beta1) * layer.dbiases
            layer.weight_cache = self.beta2 * layer.weight_cache + (1 - self.beta2) * (layer.dweights ** 2)
            layer.bias_cache = self.beta2 * layer.bias_cache + (1 - self.beta2) * (layer.dbiases ** 2)

            weight_moment_corrected = layer.weight_moment / (1 - self.beta1 ** (self.iterations + 1))
            bias_moment_corrected = layer.bias_moment / (1 - self.beta1 ** (self.iterations + 1))
            weight_cache_corrected = layer.weight_cache / (1 - self.beta2 ** (self.iterations + 1))
            bias_cache_corrected = layer.bias_cache / (1 - self.beta2 ** (self.iterations + 1))

            weight_update = -self.current_lr * weight_moment_corrected / (np.sqrt(weight_cache_corrected) + self.epsilon)
            bias_update = -self.current_lr * bias_moment_corrected / (np.sqrt(bias_cache_corrected) + self.epsilon)

        # Apply updates
        layer.weights += weight_update
        layer.biases += bias_update

    def post_update_params(self):
        self.iterations += 1


In [ ]:
import nnfs
from nnfs.datasets import spiral_data
nnfs.init()

X, y = spiral_data(samples=100, classes=3)
epochs = 1000

# ---------- TRAIN WITH OPTIMIZER 1: Learning rate decay only ----------
dense1_lr = Layer_Dense(2, 3)
activation1_lr = Activation_ReLU()
dense2_lr = Layer_Dense(3, 3)
activation2_lr = Activation_Softmax()
loss_lr = Loss_CategoricalCrossEntropy()
optimizer_lr = Optimizer_SGD(learning_rate=1.0, decay=1e-3, momentum=0.0, adaptive=False)

print("=== Training: Learning rate decay ===")
for epoch in range(epochs):
    optimizer_lr.pre_update_lr()
    dense1_lr.forward(X)
    activation1_lr.forward(dense1_lr.output)
    dense2_lr.forward(activation1_lr.output)
    activation2_lr.forward(dense2_lr.output)
    
    loss = loss_lr.calculate(activation2_lr.output, y)
    predictions = np.argmax(activation2_lr.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(f"epoch: {epoch:4d}, loss: {loss:.4f}, acc: {accuracy:.4f}, lr: {optimizer_lr.current_lr:.5f}")
    
    loss_lr.backward(activation2_lr.output, y)
    activation2_lr.backward(loss_lr.dinputs)
    dense2_lr.backward(activation2_lr.dinputs)
    activation1_lr.backward(dense2_lr.dinputs)
    dense1_lr.backward(activation1_lr.dinputs)
    
    optimizer_lr.update_params(dense1_lr)
    optimizer_lr.update_params(dense2_lr)
    optimizer_lr.post_update_params()


# ---------- TRAIN WITH OPTIMIZER 2: Momentum only ----------
dense1_mom = Layer_Dense(2, 3)
activation1_mom = Activation_ReLU()
dense2_mom = Layer_Dense(3, 3)
activation2_mom = Activation_Softmax()
loss_mom = Loss_CategoricalCrossEntropy()
optimizer_mom = Optimizer_SGD(learning_rate=0.05, decay=0.0, momentum=0.9, adaptive=False)

print("\n=== Training: Momentum ===")
for epoch in range(epochs):
    optimizer_mom.pre_update_lr()
    dense1_mom.forward(X)
    activation1_mom.forward(dense1_mom.output)
    dense2_mom.forward(activation1_mom.output)
    activation2_mom.forward(dense2_mom.output)
    
    loss = loss_mom.calculate(activation2_mom.output, y)
    predictions = np.argmax(activation2_mom.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(f"epoch: {epoch:4d}, loss: {loss:.4f}, acc: {accuracy:.4f}, lr: {optimizer_mom.current_lr:.5f}")
    
    loss_mom.backward(activation2_mom.output, y)
    activation2_mom.backward(loss_mom.dinputs)
    dense2_mom.backward(activation2_mom.dinputs)
    activation1_mom.backward(dense2_mom.dinputs)
    dense1_mom.backward(activation1_mom.dinputs)
    
    optimizer_mom.update_params(dense1_mom)
    optimizer_mom.update_params(dense2_mom)
    optimizer_mom.post_update_params()


# ---------- TRAIN WITH OPTIMIZER 3: Adaptive gradient only ----------
dense1_ad = Layer_Dense(2, 3)
activation1_ad = Activation_ReLU()
dense2_ad = Layer_Dense(3, 3)
activation2_ad = Activation_Softmax()
loss_ad = Loss_CategoricalCrossEntropy()
optimizer_ad = Optimizer_SGD(learning_rate=0.05, decay=0.0, momentum=0.0, adaptive=True)

print("\n=== Training: Adaptive gradient ===")
for epoch in range(epochs):
    optimizer_ad.pre_update_lr()
    dense1_ad.forward(X)
    activation1_ad.forward(dense1_ad.output)
    dense2_ad.forward(activation1_ad.output)
    activation2_ad.forward(dense2_ad.output)
    
    loss = loss_ad.calculate(activation2_ad.output, y)
    predictions = np.argmax(activation2_ad.output, axis=1)
    accuracy = np.mean(predictions == y)
    
    if epoch % 100 == 0 or epoch == epochs - 1:
        print(f"epoch: {epoch:4d}, loss: {loss:.4f}, acc: {accuracy:.4f}, lr: {optimizer_ad.current_lr:.5f}")
    
    loss_ad.backward(activation2_ad.output, y)
    activation2_ad.backward(loss_ad.dinputs)
    dense2_ad.backward(activation2_ad.dinputs)
    activation1_ad.backward(dense2_ad.dinputs)
    dense1_ad.backward(activation1_ad.dinputs)
    
    optimizer_ad.update_params(dense1_ad)
    optimizer_ad.update_params(dense2_ad)
    optimizer_ad.post_update_params()



=== Training: Learning rate decay only ===
epoch:    0, loss: 1.0986, acc: 0.3400, lr: 1.00000
epoch:  100, loss: 1.0983, acc: 0.3767, lr: 0.90909
epoch:  200, loss: 1.0877, acc: 0.3767, lr: 0.83333
epoch:  300, loss: 1.0836, acc: 0.4100, lr: 0.76923
epoch:  400, loss: 1.0836, acc: 0.4100, lr: 0.71429
epoch:  500, loss: 1.0835, acc: 0.4033, lr: 0.66667
epoch:  600, loss: 1.0832, acc: 0.4167, lr: 0.62500
epoch:  700, loss: 1.0825, acc: 0.4333, lr: 0.58824
epoch:  800, loss: 1.0815, acc: 0.4033, lr: 0.55556
epoch:  900, loss: 1.0809, acc: 0.3967, lr: 0.52632
epoch:  999, loss: 1.0804, acc: 0.3933, lr: 0.50025

=== Training: Momentum only ===
epoch:    0, loss: 1.0986, acc: 0.2967, lr: 0.05000
epoch:  100, loss: 1.0985, acc: 0.3900, lr: 0.05000
epoch:  200, loss: 1.0977, acc: 0.3900, lr: 0.05000
epoch:  300, loss: 1.0910, acc: 0.3900, lr: 0.05000
epoch:  400, loss: 1.0835, acc: 0.4033, lr: 0.05000
epoch:  500, loss: 1.0822, acc: 0.3967, lr: 0.05000
epoch:  600, loss: 1.0804, acc: 0.4033, 

Use most of the classes to create a functioning neural network, capable of performing a forward and backward pass

We can use a sample dataset from the Spiral module.  

We can also use the IRIS dataset.

In [13]:
# Spiral Data
import nnfs
from nnfs.datasets import spiral_data

# Create the dataset
X, y = spiral_data(samples = 100, classes = 3)

# print(X[:5])
# print(X.shape)
# print(y.shape)
# print(y[:5])

ModuleNotFoundError: No module named 'nnfs'

In [ ]:
# Iris Dataset
# From the scikit-learn library
# from sklearn.datasets import load_iris
# iris = load_iris()
# X = iris.data # Features
# y = iris.target # Target labels

# print(X[:5])
# print(X.shape)
# print(y[:5])
# print(y.shape)

In [ ]:
# Neural Network initialization
# Create a Dense Layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Make sure you check the shape of the features, in order to adjust the input size of the first layer
# dense1 = Layer_Dense(4, 3)

# Create a ReLU activation for the first Dense layer
activation1 = Activation_ReLU()

# Create a 2nd dense layer with 3 input and 3 output values
dense2 = Layer_Dense(3, 3)

# Create a Softmax activation for the 2nd Dense layer
activation2 = Activation_Softmax()

# Create a loss function
loss_function = Loss_CategoricalCrossEntropy()

# Create the optimizer
optimizer = Optimizer_SGD()

PERFORM ONLY 1 PASS

In [ ]:
# Perform a forward pass of our training data
# give the input from the dataset to the first layer
dense1.forward(X)

# Activation function
activation1.forward(dense1.output)

# Pass on the 2nd layer
dense2.forward(activation1.output)

activation2.forward(dense2.output)

# Calculate the loss
loss_function.forward(activation2.output, y)

# Check the model's performance
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

# Print the accuracy
print('acc:', accuracy)

In [ ]:
# Perform a backward pass of our training data
# From loss to 2nd softmax activation
loss_function.backward(activation2.output, y)
dvalues = loss_function.dinputs # Gradient of the loss w.r.t softmax output

print(dvalues.shape)
# print(dvalues)

# From 2nd softmax to 2nd dense layer
activation2.backward(dvalues)
# From 2nd dense layer to 1st ReLU activation
dense2.backward(activation2.dinputs)

# From 1st ReLU activation to 1st dense layer
activation1.backward(dense2.dinputs)
dense1.backward(activation1.dinputs)

In [ ]:
# Check the gradient values of the weights and biases of the established layers
print(dense1.dweights)
print(dense1.dbiases)
print(dense2.dweights)
print(dense2.dbiases)


# Update the weights and biases
optimizer.update_params(dense1)
optimizer.update_params(dense2)